In [13]:
import numpy as np
import pandapower as pp
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 8]

# Definición de la red y cálculo de Ybus
P = 150
Q = 100

net = pp.create_empty_network()

# Crear barras
bus1 = pp.create_bus(net, vn_kv=110, name="Barra 1")
bus2 = pp.create_bus(net, vn_kv=220, name="Barra 2")
bus1a = pp.create_bus(net, vn_kv=220, name="Barra 1A")
bus2a = pp.create_bus(net, vn_kv=220, name="Barra 2A")
bus3a = pp.create_bus(net, vn_kv=220, name="Barra 3A")
bus1b = pp.create_bus(net, vn_kv=220, name="Barra 1B")
bus2b = pp.create_bus(net, vn_kv=220, name="Barra 2B")

# Crear transformador entre bus 1 y bus 2
pp.create_transformer(net, hv_bus=bus2, lv_bus=bus1, std_type="100 MVA 220/110 kV")

# Crear el generador en bus1 y definirlo como la barra slack
pp.create_ext_grid(net, bus=bus1, vm_pu=1.0, va_degree=0, name="Barra Slack")

# Crear cargas entre barras
pp.create_load(net, bus1a, p_mw=0.2*P, q_mvar=0.2*Q, name="Carga 1A")
pp.create_load(net, bus2a, p_mw=0.35*P, q_mvar=0.35*Q, name="Carga 2A")
pp.create_load(net, bus3a, p_mw=0.15*P, q_mvar=0.15*Q, name="Carga 3A")
pp.create_load(net, bus1b, p_mw=0.1*P, q_mvar=0.1*Q, name="Carga 1B")
pp.create_load(net, bus2b, p_mw=0.6*P, q_mvar=0.6*Q, name="Carga 2B")

# Crear líneas entre barras
pp.create_line(net, from_bus=bus2, to_bus=bus1a, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name='L21A', in_service=True)
pp.create_line(net, from_bus=bus1a, to_bus=bus2a, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name='L1A2A', in_service=True)
pp.create_line(net, from_bus=bus2a, to_bus=bus3a, length_km=20, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name='L2A3A', in_service=True)
pp.create_line(net, from_bus=bus3a, to_bus=bus2b, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name='L3A2B', in_service=True)
pp.create_line(net, from_bus=bus2, to_bus=bus1b, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name='L21B', in_service=True)
pp.create_line(net, from_bus=bus1b, to_bus=bus2b, length_km=30, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name='L1B2B', in_service=True)

# Ejecutar el flujo de carga
pp.runpp(net)
Ybus = net._ppc['internal']['Ybus']

# Dividir todos los valores de la matriz Ybus por 100
Ybus100 = Ybus / 100

# Convertir Ybus a un DataFrame de pandas para una mejor visualización
Ybus_df = pd.DataFrame(data=Ybus100.todense())
print("Matriz Ybus (7x7) ajustada:")
print(Ybus_df)

# Definición de la función de Newton-Raphson
def F(x):
    V = x[:len(x)//2]
    theta = x[len(x)//2:]
    
    P = np.zeros(len(V))
    Q = np.zeros(len(V))
    
    for i in range(len(V)):
        for j in range(len(V)):
            G = Ybus[i, j].real
            B = Ybus[i, j].imag
            P[i] += V[i] * V[j] * (G * np.cos(theta[i] - theta[j]) + B * np.sin(theta[i] - theta[j]))
            Q[i] += V[i] * V[j] * (G * np.sin(theta[i] - theta[j]) - B * np.cos(theta[i] - theta[j]))
    
    P_spec = net.res_bus.p_mw.values / net.sn_mva
    Q_spec = net.res_bus.q_mvar.values / net.sn_mva

    # Excluir barra slack de las ecuaciones
    P[0] = 0
    Q[0] = 0
    P_spec[0] = 0
    Q_spec[0] = 0
    
    return np.concatenate([P - P_spec, Q - Q_spec])

# Derivación del Jacobiano
def J(x):
    V = x[:len(x)//2]
    theta = x[len(x)//2:]
    
    J11 = np.zeros((len(V), len(V)))
    J12 = np.zeros((len(V), len(V)))
    J21 = np.zeros((len(V), len(V)))
    J22 = np.zeros((len(V), len(V)))
    
    for i in range(len(V)):
        for j in range(len(V)):
            G = Ybus[i, j].real
            B = Ybus[i, j].imag
            
            if i == j:
                for k in range(len(V)):
                    if k != i:
                        J11[i, j] += V[i] * V[k] * (-G * np.sin(theta[i] - theta[k]) + B * np.cos(theta[i] - theta[k]))
                        J12[i, j] += V[k] * (G * np.cos(theta[i] - theta[k]) + B * np.sin(theta[i] - theta[k]))
                        J21[i, j] += V[i] * V[k] * (G * np.cos(theta[i] - theta[k]) + B * np.sin(theta[i] - theta[k]))
                        J22[i, j] += V[k] * (G * np.sin(theta[i] - theta[k]) - B * np.cos(theta[i] - theta[k]))
                J11[i, j] += V[i] * Ybus[i, i].imag
                J12[i, j] += V[i] * Ybus[i, i].real
                J21[i, j] += V[i] * (-Ybus[i, i].real)
                J22[i, j] += V[i] * Ybus[i, i].imag
            else:
                J11[i, j] = V[i] * V[j] * (G * np.sin(theta[i] - theta[j]) - B * np.cos(theta[i] - theta[j]))
                J12[i, j] = V[i] * (G * np.cos(theta[i] - theta[j]) + B * np.sin(theta[i] - theta[j]))
                J21[i, j] = V[i] * V[j] * (-G * np.cos(theta[i] - theta[j]) - B * np.sin(theta[i] - theta[j]))
                J22[i, j] = V[i] * (G * np.sin(theta[i] - theta[j]) - B * np.cos(theta[i] - theta[j]))
    
    return np.block([[J11, J12], [J21, J22]])

# Implementación del método de Newton-Raphson
def newton_raphson(F, J, x0, tol=1e-6, max_iter=100):
    x = x0
    for i in range(max_iter):
        Jx = J(x)
        Fx = F(x)
        delta = np.linalg.solve(Jx, -Fx)
        x = x + delta
        if np.linalg.norm(delta, np.inf) < tol:
            print(f'Convergencia alcanzada en {i+1} iteraciones.')
            break
    else:
        print('No se alcanzó la convergencia.')
    return x

# Vector inicial (V y θ)
V0 = net.res_bus.vm_pu.values
theta0 = net.res_bus.va_degree.values * np.pi / 180
x0 = np.concatenate([V0, theta0])

# Ejecutar Newton-Raphson
resultado = newton_raphson(F, J, x0)

# Resultados
V_final = resultado[:len(resultado)//2]
theta_final = resultado[len(resultado)//2:] * 180 / np.pi
print("Tensiones (pu):", V_final)
print("Ángulos (grados):", theta_final)


Matriz Ybus (7x7) ajustada:
                    0                       1                       2  \
0  0.180693-8.331437j   -0.180418+  8.331317j    0.000000+  0.000000j   
1 -0.180418+8.331317j  280.908349-450.500102j -140.363828+221.179365j   
2  0.000000+0.000000j -140.363828+221.179365j  233.939714-368.394693j   
3  0.000000+0.000000j    0.000000+  0.000000j  -93.575885+147.452910j   
4  0.000000+0.000000j    0.000000+  0.000000j    0.000000+  0.000000j   
5  0.000000+0.000000j -140.363828+221.179365j    0.000000+  0.000000j   
6  0.000000+0.000000j    0.000000+  0.000000j    0.000000+  0.000000j   

                        3                       4                       5  \
0    0.000000+  0.000000j    0.000000+  0.000000j    0.000000+  0.000000j   
1    0.000000+  0.000000j    0.000000+  0.000000j -140.363828+221.179365j   
2  -93.575885+147.452910j    0.000000+  0.000000j    0.000000+  0.000000j   
3  163.757799-257.709977j  -70.181914+110.589683j    0.000000+  0.000000j   
4 